# **Anchor google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## **Unzip files**

In [ ]:
zip_path = "/content/drive/MyDrive/AD_SMART_CITY/Input/data.zip"
out_path = "/content/drive/MyDrive/AD_SMART_CITY/Input/"

!unzip "$zip_path" -d "$out_path"

!rm -f "$zip_path"

Output streaming troncato alle ultime 5000 righe.
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_17/normal/day_7.csv  
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_17/normal/day_8.csv  
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_17/normal/day_9.csv  
   creating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/
   creating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/anomalous/
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/anomalous/day_17.csv  
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/anomalous/day_28.csv  
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/anomalous/day_29.csv  
  inflating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/anomalous/day_4.csv  
   creating: /content/drive/MyDrive/AD_SMART_CITY/Input/faulty_raw/2.5/TF_18/normal/
  inflating:

# **Import libraries**

In [ ]:
import pytz
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve, auc
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
import copy
from tqdm.notebook import tqdm
import torch.optim as optim
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor



# **Global definitions**

In [ ]:
input_dir = "/content/drive/MyDrive/AD_SMART_CITY/Input/"
output_dir = "/content/drive/MyDrive/AD_SMART_CITY/Output/"
output_metrics_dir = output_dir + "Metrics/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# **Load the data**

In [ ]:
def read_csv(file_path, expected_cols=None, timestamp_unit='s'):
    try:
        df = pd.read_csv(file_path)
        if expected_cols:
            df = df[[col for col in expected_cols if col in df.columns]]
        if "timestamp" in df.columns:
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit=timestamp_unit)
            df.sort_values("timestamp", inplace=True)
            df.reset_index(drop=True, inplace=True)
        return df
    except Exception as e:
        print(f"    ⚠️ Error reading {file_path}: {e}")
        return None

def load_csvs_as_days(folder_path, expected_cols=None, max_workers=4, timestamp_unit='s'):
    if not os.path.exists(folder_path):
        return []

    csv_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")])
    if not csv_files:
        return []

    print(f"    📁 Found {len(csv_files)} CSVs in {os.path.basename(folder_path)}")

    day_dfs = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(lambda f: read_csv(f, expected_cols, timestamp_unit), csv_files)
        day_dfs = [df for df in results if df is not None]

    return day_dfs

def load_dataset_hierarchical_per_day(
    base_dir,
    expected_cols=None,
    split_sensors=False,
    max_workers=4,
    selected_granularities=None
):
    try:
        all_gran_folders = sorted([f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))])
    except FileNotFoundError:
        print(f"[ERROR] Base directory not found: {base_dir}")
        return {}

    if selected_granularities is not None:
        selected_set = set(str(g) for g in selected_granularities)
        granularity_folders = [f for f in all_gran_folders if f in selected_set]
    else:
        granularity_folders = all_gran_folders

    print(f"  Processing {len(granularity_folders)} granularities: {granularity_folders}")

    if split_sensors:
        sensor_data = {f"sensor_{i}": {} for i in range(4)}
    else:
        data_dict = {}

    for gran in granularity_folders:
        print(f"   Loading Granularity: {gran}")
        gran_path = os.path.join(base_dir, gran)
        tf_folders = sorted([f for f in os.listdir(gran_path) if f.startswith("TF_")])

        if split_sensors:
            for i in range(4):
                sensor_data[f"sensor_{i}"][gran] = {}
        else:
            data_dict[gran] = {}

        for tf in tf_folders:
            tf_path = os.path.join(gran_path, tf)
            normal_path = os.path.join(tf_path, "normal")
            anomalous_path = os.path.join(tf_path, "anomalous")

            normal_days = load_csvs_as_days(normal_path, expected_cols, max_workers)
            anomalous_days = load_csvs_as_days(anomalous_path, expected_cols, max_workers)

            if split_sensors:
                for i in range(4):
                    col = f"cal_pm25_{i}"
                    sensor_data[f"sensor_{i}"][gran][tf] = {
                        "normal": [df[["timestamp", col]] for df in normal_days if col in df.columns],
                        "anomalous": [df[["timestamp", col]] for df in anomalous_days if col in df.columns]
                    }
            else:
                data_dict[gran][tf] = {"normal": normal_days, "anomalous": anomalous_days}

    return sensor_data if split_sensors else data_dict

SELECTED_GRANULARITIES = ['2.5', '5', '10', '20', '25', '50']

print("=== Loading fault_free ===")
fault_free = load_dataset_hierarchical_per_day(
    os.path.join(input_dir, "fault_free"),
    expected_cols=["timestamp", "cal_pm25_fa"],
    split_sensors=False,
    max_workers=8,
    selected_granularities=SELECTED_GRANULARITIES
)

print("\n=== Loading faulty_fa ===")
faulty_fa = load_dataset_hierarchical_per_day(
    os.path.join(input_dir, "faulty_fa"),
    expected_cols=["timestamp", "cal_pm25_fa"],
    split_sensors=False,
    max_workers=8,
    selected_granularities=SELECTED_GRANULARITIES
)

print("\n=== Loading faulty_raw ===")
faulty_raw = load_dataset_hierarchical_per_day(
    os.path.join(input_dir, "faulty_raw"),
    expected_cols=["timestamp", "cal_pm25_0", "cal_pm25_1", "cal_pm25_2", "cal_pm25_3"],
    split_sensors=True,
    max_workers=8,
    selected_granularities=SELECTED_GRANULARITIES
)

=== Loading fault_free ===
  Processing 6 granularities: ['10', '2.5', '20', '25', '5', '50']
   Loading Granularity: 10
    📁 Found 25 CSVs in normal
    📁 Found 7 CSVs in anomalous
    📁 Found 30 CSVs in normal
    📁 Found 28 CSVs in normal
    📁 Found 2 CSVs in anomalous
    📁 Found 28 CSVs in normal
    📁 Found 5 CSVs in anomalous
    📁 Found 27 CSVs in normal
    📁 Found 5 CSVs in anomalous
    📁 Found 30 CSVs in normal
    📁 Found 1 CSVs in anomalous
    📁 Found 21 CSVs in normal
    📁 Found 13 CSVs in anomalous
    📁 Found 26 CSVs in normal
    📁 Found 6 CSVs in anomalous
    📁 Found 25 CSVs in normal
    📁 Found 7 CSVs in anomalous
    📁 Found 29 CSVs in normal
    📁 Found 2 CSVs in anomalous
   Loading Granularity: 2.5
    📁 Found 27 CSVs in normal
    📁 Found 3 CSVs in anomalous
    📁 Found 26 CSVs in normal
    📁 Found 7 CSVs in anomalous
    📁 Found 27 CSVs in normal
    📁 Found 4 CSVs in anomalous
    📁 Found 28 CSVs in normal
    📁 Found 4 CSVs in anomalous
    📁 Found 29

# **Dataset summary**

In [ ]:
def print_dataset_summary(dataset, dataset_name):
    print(f"\n=== {dataset_name} dataset summary ===")

    if dataset_name == "faulty_raw":
        for sensor, gran_dict in dataset.items():
            print(f"\nSensor: {sensor}")
            for gran, tf_dict in gran_dict.items():
                for tf, day_dict in tf_dict.items():
                    n_normal = len(day_dict.get("normal", []))
                    n_anomalous = len(day_dict.get("anomalous", []))
                    print(f"  Granularity {gran}, TF {tf}: {n_normal} normal, {n_anomalous} anomalous days")
    else:
        for gran, tf_dict in dataset.items():
            for tf, day_dict in tf_dict.items():
                n_normal = len(day_dict.get("normal", []))
                n_anomalous = len(day_dict.get("anomalous", []))
                print(f"  Granularity {gran}, TF {tf}: {n_normal} normal, {n_anomalous} anomalous days")

print_dataset_summary(fault_free, "fault_free")
print_dataset_summary(faulty_fa, "faulty_fa")
print_dataset_summary(faulty_raw, "faulty_raw")


=== fault_free dataset summary ===
  Granularity 10, TF TF_0: 27 normal, 3 anomalous days
  Granularity 10, TF TF_1: 28 normal, 2 anomalous days
  Granularity 10, TF TF_2: 27 normal, 3 anomalous days
  Granularity 10, TF TF_3: 28 normal, 2 anomalous days
  Granularity 10, TF TF_4: 24 normal, 6 anomalous days
  Granularity 10, TF TF_5: 29 normal, 1 anomalous days
  Granularity 10, TF TF_6: 29 normal, 1 anomalous days
  Granularity 10, TF TF_7: 30 normal, 0 anomalous days
  Granularity 10, TF TF_8: 27 normal, 3 anomalous days
  Granularity 10, TF TF_9: 28 normal, 2 anomalous days
  Granularity 2.5, TF TF_0: 28 normal, 1 anomalous days
  Granularity 2.5, TF TF_1: 26 normal, 4 anomalous days
  Granularity 2.5, TF TF_10: 25 normal, 5 anomalous days
  Granularity 2.5, TF TF_11: 26 normal, 4 anomalous days
  Granularity 2.5, TF TF_12: 28 normal, 2 anomalous days
  Granularity 2.5, TF TF_13: 27 normal, 3 anomalous days
  Granularity 2.5, TF TF_14: 28 normal, 2 anomalous days
  Granularity 2.5

# **Training and test set building**

In [ ]:
TRAINING_STEP_LENGTH = 10
TEST_STEP_LENGTH = 1
SEQ_LENGTH = 10
FEATURE_COLS = ['cal_pm25_fa']

def split_and_synchronize_datasets(
    fault_free_data,
    faulty_fa_data,
    faulty_raw_data,
    train_ratio,
    seed=42
):
    random.seed(seed)
    training_data = {}
    print(f"{'='*20} STARTING DATASET SPLIT (Seed: {seed}) {'='*20}")

    for gran in fault_free_data.keys():
        if gran not in training_data:
            training_data[gran] = {}
        for tf in fault_free_data[gran].keys():
            training_data[gran][tf] = []
            initial_counts = {}
            if 'normal' in fault_free_data[gran][tf]:
                 initial_counts['Fault Free'] = len(fault_free_data[gran][tf]['normal'])
            else:
                 initial_counts['Fault Free'] = 0

            if gran in faulty_fa_data and tf in faulty_fa_data[gran] and 'normal' in faulty_fa_data[gran][tf]:
                initial_counts['Faulty FA'] = len(faulty_fa_data[gran][tf]['normal'])

            for sensor in faulty_raw_data.keys():
                if gran in faulty_raw_data[sensor] and tf in faulty_raw_data[sensor][gran] and 'normal' in faulty_raw_data[sensor][gran][tf]:
                    initial_counts[f'Raw {sensor}'] = len(faulty_raw_data[sensor][gran][tf]['normal'])

            if 'normal' not in fault_free_data[gran][tf] or not fault_free_data[gran][tf]['normal']:
                continue

            ff_normal_list = fault_free_data[gran][tf]['normal']
            total_days = len(ff_normal_list)
            n_train_days = int(total_days * train_ratio)

            if n_train_days == 0:
                continue

            all_indices = list(range(total_days))
            train_indices = set(random.sample(all_indices, n_train_days))

            for idx in train_indices:
                training_data[gran][tf].append(ff_normal_list[idx])

            def filter_indices(data_list):
                return [d for i, d in enumerate(data_list) if i not in train_indices]

            fault_free_data[gran][tf]['normal'] = filter_indices(ff_normal_list)

            if 'Faulty FA' in initial_counts:
                faulty_fa_data[gran][tf]['normal'] = filter_indices(faulty_fa_data[gran][tf]['normal'])

            for sensor in faulty_raw_data.keys():
                if f'Raw {sensor}' in initial_counts:
                    faulty_raw_data[sensor][gran][tf]['normal'] = filter_indices(faulty_raw_data[sensor][gran][tf]['normal'])


    print(f"{'='*20} SPLIT COMPLETE (Seed: {seed}) {'='*20}")
    return training_data, fault_free_data, faulty_fa_data, faulty_raw_data

class AnomalyDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        seq = torch.tensor(self.sequences[idx], dtype=torch.float32)
        return seq, seq

def create_sequences(data, seq_length, step=1):
    xs = []
    if len(data) <= seq_length:
        return np.array(xs)
    for i in range(0, len(data) - seq_length + 1, step):
        x = data[i:(i + seq_length)]
        xs.append(x)
    return np.array(xs)

def build_training_registry(training_data_dict, feature_cols, seq_len):
    registry = {}
    print(f"{'='*20} BUILDING MODEL REGISTRY {'='*20}")
    INTERNAL_LOADER_BATCH_SIZE = 32

    for gran in training_data_dict.keys():
        for tf in training_data_dict[gran].keys():
            day_dfs = training_data_dict[gran][tf]
            if not day_dfs:
                continue

            full_tf_df = pd.concat(day_dfs, ignore_index=True)
            if full_tf_df.empty:
                continue

            local_scaler = MinMaxScaler()
            local_scaler.fit(full_tf_df[feature_cols])

            tf_sequences = []
            for day_df in day_dfs:
                scaled_data = local_scaler.transform(day_df[feature_cols])
                seqs = create_sequences(scaled_data, seq_len, step=TRAINING_STEP_LENGTH)
                if len(seqs) > 0:
                    tf_sequences.append(seqs)

            if len(tf_sequences) > 0:
                X_train_local = np.concatenate(tf_sequences, axis=0)
                dataset = AnomalyDataset(X_train_local)
                loader = DataLoader(dataset, batch_size=INTERNAL_LOADER_BATCH_SIZE, shuffle=True, drop_last=True)
                registry[(gran, tf)] = {
                    "loader": loader,
                    "scaler": local_scaler,
                    "n_samples": len(X_train_local)
                }
    print(f"{'='*20} REGISTRY COMPLETE ({len(registry)} pairs) {'='*20}")
    return registry

def build_specialized_test_registry(
    ff_data,
    ffa_data,
    fraw_data,
    model_registry,
    train_feature_col,
    seq_len
):
    test_registry = {}
    stats = {"processed": 0, "skipped": 0}
    print(f"{'='*20} BUILDING SPECIALIZED TEST REGISTRY {'='*20}")

    def process_and_store(df_list, gran, tf, source_col_name, label, category):
        if (gran, tf) not in model_registry:
            return 0
        local_scaler = model_registry[(gran, tf)]['scaler']
        if (gran, tf) not in test_registry:
            test_registry[(gran, tf)] = []

        count = 0
        for df in df_list:
            temp_df = df[[source_col_name]].copy()
            if source_col_name != train_feature_col:
                temp_df.rename(columns={source_col_name: train_feature_col}, inplace=True)

            data_scaled = local_scaler.transform(temp_df)
            sequences = create_sequences(data_scaled, seq_len, step=TEST_STEP_LENGTH)

            if len(sequences) > 0:
                tensor_data = torch.tensor(sequences, dtype=torch.float32)
                day_obj = {
                    "data": tensor_data, "label": label, "category": category,
                    "meta": {"gran": gran, "tf": tf}
                }
                test_registry[(gran, tf)].append(day_obj)
                count += 1
        return count

    for gran in ff_data:
        for tf in ff_data[gran]:
            if 'normal' in ff_data[gran][tf]:
                stats["processed"] += process_and_store(ff_data[gran][tf]['normal'], gran, tf, train_feature_col, 0, "FF_Normal")
            if 'anomalous' in ff_data[gran][tf]:
                stats["processed"] += process_and_store(ff_data[gran][tf]['anomalous'], gran, tf, train_feature_col, 1, "FF_Anom")

    for gran in ffa_data:
        for tf in ffa_data[gran]:
            if 'normal' in ffa_data[gran][tf]:
                stats["processed"] += process_and_store(ffa_data[gran][tf]['normal'], gran, tf, train_feature_col, 0, "FA_Normal")
            if 'anomalous' in ffa_data[gran][tf]:
                stats["processed"] += process_and_store(ffa_data[gran][tf]['anomalous'], gran, tf, train_feature_col, 1, "FA_Anom")

    for sensor_key in fraw_data:
        try:
            sensor_id = sensor_key.split('_')[1]
            raw_col_name = f"cal_pm25_{sensor_id}"
        except IndexError:
            continue
        for gran in fraw_data[sensor_key]:
            for tf in fraw_data[sensor_key][gran]:
                if 'normal' in fraw_data[sensor_key][gran][tf]:
                    stats["processed"] += process_and_store(fraw_data[sensor_key][gran][tf]['normal'], gran, tf, raw_col_name, 0, f"Raw_{sensor_key}_Normal")
                if 'anomalous' in fraw_data[sensor_key][gran][tf]:
                    stats["processed"] += process_and_store(fraw_data[sensor_key][gran][tf]['anomalous'], gran, tf, raw_col_name, 1, f"Raw_{sensor_key}_Anom")

    print(f"Test Registry Complete.")
    print(f"  > Total Test Days Processed: {stats['processed']}")
    return test_registry

N_REPETITIONS = 5
TRAIN_RATIO = 0.75

all_model_registries = []
all_test_registries = []

if 'fault_free' not in locals():
    print("ERROR: Original datasets (fault_free, faulty_fa, faulty_raw) not found.")
    print("Please load the data in a cell above this one.")
else:
    for i in range(N_REPETITIONS):
        print(f"\n{'='*40}")
        print(f"  STARTING REPETITION {i + 1} / {N_REPETITIONS}")
        print(f"{'='*40}\n")

        ff_copy = copy.deepcopy(fault_free)
        ffa_copy = copy.deepcopy(faulty_fa)
        fraw_copy = copy.deepcopy(faulty_raw)

        train_set, ff_clean, ffa_clean, fraw_clean = split_and_synchronize_datasets(
            ff_copy,
            ffa_copy,
            fraw_copy,
            train_ratio=TRAIN_RATIO,
            seed=i
        )

        model_registry = build_training_registry(
            train_set,
            FEATURE_COLS,
            SEQ_LENGTH
        )

        test_registry = build_specialized_test_registry(
            ff_clean,
            ffa_clean,
            fraw_clean,
            model_registry,
            "cal_pm25_fa",
            SEQ_LENGTH
        )

        all_model_registries.append(model_registry)
        all_test_registries.append(test_registry)

    print(f"\n\n{'='*40}")
    print(f"  COMPLETED {N_REPETITIONS} REPETITIONS")
    print(f"  Created {len(all_model_registries)} model registries.")
    print(f"  Created {len(all_test_registries)} test registries.")
    print(f"{'='*40}")


  STARTING REPETITION 1 / 5

==================== STARTING DATASET SPLIT (Seed: 0) ====================
==================== SPLIT COMPLETE (Seed: 0) ====================
==================== BUILDING MODEL REGISTRY ====================
==================== REGISTRY COMPLETE (81 pairs) ====================
==================== BUILDING SPECIALIZED TEST REGISTRY ====================
Test Registry Complete.
  > Total Test Days Processed: 5754

  STARTING REPETITION 2 / 5

==================== STARTING DATASET SPLIT (Seed: 1) ====================
==================== SPLIT COMPLETE (Seed: 1) ====================
==================== BUILDING MODEL REGISTRY ====================
==================== REGISTRY COMPLETE (81 pairs) ====================
==================== BUILDING SPECIALIZED TEST REGISTRY ====================
Test Registry Complete.
  > Total Test Days Processed: 5754

  STARTING REPETITION 3 / 5

==================== STARTING DATASET SPLIT (Seed: 2) ====================
===

In [ ]:
def print_registry_stats(registry):
    print(f"\n{'='*20} TEST REGISTRY STRUCTURE REPORT {'='*20}")

    if not registry:
        print("Registry is empty!")
        return

    try:
        sorted_keys = sorted(registry.keys(), key=lambda x: (x[0], int(x[1].split('_')[1])))
    except:
        sorted_keys = sorted(registry.keys())

    total_global_days = 0

    for gran, tf in sorted_keys:
        day_list = registry[(gran, tf)]
        n_total = len(day_list)
        total_global_days += n_total

        n_normal = sum(1 for d in day_list if d['label'] == 0)
        n_anomalous = sum(1 for d in day_list if d['label'] == 1)

        cat_counts = {}
        for d in day_list:
            cat = d['category']
            cat_counts[cat] = cat_counts.get(cat, 0) + 1

        print(f"Key: [{gran} | {tf:<5}]")
        print(f"  ├── Total Days: {n_total}")
        print(f"  ├── Labels    : {n_normal} Normal (0) | {n_anomalous} Anomalous (1)")
        print(f"  └── Sources   :")

        for cat in sorted(cat_counts.keys()):
            print(f"      • {cat:<25}: {cat_counts[cat]}")

        print("-" * 50)

    print(f"TOTAL TEST SAMPLES ACROSS ALL PAIRS: {total_global_days}")
    print(f"{'='*50}\n")

print_registry_stats(test_registry)

# **LSTM-AE**

In [ ]:
N_FEATURES = len(FEATURE_COLS)
HIDDEN_DIM = 16
N_EPOCHS = 50
BATCH_SIZE = 512
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_OUTPUT_DIR = output_metrics_dir + "LSTM-AE"

print(f"Using device: {DEVICE}")

class LSTM_AE(nn.Module):
    def __init__(self, n_features, seq_len, hidden_dim):
        super(LSTM_AE, self).__init__()
        self.seq_len = seq_len
        self.n_features = n_features
        self.hidden_dim = hidden_dim

        self.encoder = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.decoder = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.output_layer = nn.Linear(hidden_dim, n_features)

    def forward(self, x):
        _, (hidden, _) = self.encoder(x)
        decoder_input = hidden.squeeze(0).unsqueeze(1).repeat(1, self.seq_len, 1)
        dec_out, _ = self.decoder(decoder_input)
        reconstruction = self.output_layer(dec_out)
        return reconstruction

def extract_anomaly_scores(trained_models_dict, test_registry_dict, device):
    results_list = []
    print(f"{'='*20} STARTING SCORE EXTRACTION {'='*20}")

    for key, test_days in tqdm(test_registry_dict.items(), desc="Scoring Test Days", leave=False):
        gran, tf = key
        if key not in trained_models_dict:
            continue

        bundle = trained_models_dict[key]
        model = bundle['model'].to(device).eval()
        for day_idx, day_obj in enumerate(test_days):
            inputs = day_obj['data'].to(device)
            if len(inputs) == 0:
                continue
            with torch.no_grad():
                reconstructions = model(inputs)
                mse_per_seq = torch.mean((inputs - reconstructions)**2, dim=(1, 2))
                day_score = torch.mean(mse_per_seq).item()
            results_list.append({
                "granularity": gran, "time_frame": tf, "day_index": day_idx,
                "ad_score": day_score, "true_label": day_obj['label'],
                "category": day_obj['category']
            })
    df_results = pd.DataFrame(results_list)
    print(f"Extraction Complete. Processed {len(df_results)} days.")
    return df_results

def calculate_metrics_at_threshold(y_true, y_scores, thresh):
    y_pred = (y_scores >= thresh).astype(int)
    return {
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "Accuracy": accuracy_score(y_true, y_pred)
    }

def analyze_all_metrics(df_results, output_dir: str):
    if df_results.empty:
        print("DataFrame is empty.")
        return None

    os.makedirs(output_dir, exist_ok=True)
    roc_data_dir = os.path.join(output_dir, "roc_curve_data")
    os.makedirs(roc_data_dir, exist_ok=True)
    print(f"Metrics and ROC data will be saved to {output_dir}")

    subgroups = {
        "Fault Free":   lambda x: "FF" in x, "Faulty (F+A)": lambda x: "FA" in x,
        "Sensor 1":     lambda x: "sensor_0" in x, "Sensor 2":     lambda x: "sensor_1" in x,
        "Sensor 3":     lambda x: "sensor_2" in x, "Sensor 4":     lambda x: "sensor_3" in x,
    }
    metrics_list = []
    unique_grans = sorted(df_results['granularity'].unique())

    print(f"{'='*20} DETAILED METRIC ANALYSIS BY GRANULARITY {'='*20}")
    for group_name, filter_func in subgroups.items():
        for gran in unique_grans:
            subset = df_results[
                (df_results['granularity'] == gran) &
                (df_results['category'].apply(filter_func))
            ]
            if len(subset) == 0:
                continue

            y_true, y_scores = subset['true_label'], subset['ad_score']

            auc = np.nan
            metrics_f1, metrics_roc, metrics_rec90, metrics_fpr10 = [
                {"F1": np.nan, "Precision": np.nan, "Recall": np.nan, "Accuracy": np.nan}
            ] * 4

            if len(np.unique(y_true)) > 1:
                auc = roc_auc_score(y_true, y_scores)
                fpr, tpr, roc_threshs = roc_curve(y_true, y_scores)
                pr, re, pr_threshs = precision_recall_curve(y_true, y_scores)

                # --- 1. Best F1-Score Threshold ---
                f1_scores = np.divide(2 * pr * re, pr + re, out=np.zeros_like(pr), where=(pr + re) != 0)
                best_f1_idx = np.argmax(f1_scores)
                thresh_best_f1 = pr_threshs[min(best_f1_idx, len(pr_threshs)-1)]
                metrics_f1 = calculate_metrics_at_threshold(y_true, y_scores, thresh_best_f1)

                # --- 2. Best ROC (Top-Left) Threshold ---
                dist = (fpr - 0)**2 + (tpr - 1)**2
                best_roc_idx = np.argmin(dist)
                thresh_roc_topleft = roc_threshs[best_roc_idx]
                metrics_roc = calculate_metrics_at_threshold(y_true, y_scores, thresh_roc_topleft)

                # --- 3. Fixed Recall @ 90% Threshold ---
                recall_90_indices = np.where(tpr >= 0.90)[0]
                if len(recall_90_indices) > 0:
                    thresh_recall_90 = roc_threshs[recall_90_indices[0]] # First thresh to hit 90%
                    metrics_rec90 = calculate_metrics_at_threshold(y_true, y_scores, thresh_recall_90)

                # --- 4. Fixed FPR @ 10% Threshold ---
                fpr_10_indices = np.where(fpr <= 0.10)[0]
                if len(fpr_10_indices) > 0:
                    thresh_fpr_10 = roc_threshs[fpr_10_indices[-1]] # Last thresh to be <= 10%
                    metrics_fpr10 = calculate_metrics_at_threshold(y_true, y_scores, thresh_fpr_10)

                try:
                    roc_data_df = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'thresholds': roc_threshs})
                    safe_group_name = group_name.replace(" ", "_").replace("+", "").replace("(", "").replace(")", "")
                    file_name = f"roc_{safe_group_name}_{str(gran)}.csv"
                    roc_data_df.to_csv(os.path.join(roc_data_dir, file_name), index=False)
                except Exception as e:
                    print(f"Warning: Could not save ROC data for {group_name}/{gran}. Error: {e}")

            row = {
                "Dataset": group_name, "Granularity": gran, "AUC": auc,
                "Total Days": len(subset), "Normal": sum(y_true == 0), "Anom": sum(y_true == 1)
            }
            for prefix, met_dict in [("BestF1", metrics_f1), ("ROC_TopLeft", metrics_roc),
                                     ("Recall_90", metrics_rec90), ("FPR_10", metrics_fpr10)]:
                for met_name, met_val in met_dict.items():
                    row[f"{met_name}_{prefix}"] = met_val

            metrics_list.append(row)

    summary_df = pd.DataFrame(metrics_list)

    metrics_to_pivot = [
        "AUC",
        "F1_BestF1", "Precision_BestF1", "Recall_BestF1", "Accuracy_BestF1",
        "F1_ROC_TopLeft", "Precision_ROC_TopLeft", "Recall_ROC_TopLeft", "Accuracy_ROC_TopLeft",
        "F1_Recall_90", "Precision_Recall_90", "Recall_Recall_90", "Accuracy_Recall_90",
        "F1_FPR_10", "Precision_FPR_10", "Recall_FPR_10", "Accuracy_FPR_10"
    ]
    metrics_to_print = ["AUC", "F1_BestF1", "F1_ROC_TopLeft", "F1_Recall_90", "F1_FPR_10"]

    pivot_tables = {}

    for metric in metrics_to_pivot:
        if metric in summary_df.columns:
            pivot = summary_df.pivot(index="Dataset", columns="Granularity", values=metric)
            pivot_tables[metric] = pivot

            if metric in metrics_to_print:
                print(f"\n--- {metric} Matrix (Rows=Dataset, Cols=Granularity) ---")
                print(pivot)

    try:
        summary_path = os.path.join(output_dir, "metrics_summary_long.csv")
        summary_df.to_csv(summary_path, index=False)
        print(f"\nSuccessfully saved LONG summary to {output_dir}")

        for metric, pivot_df in pivot_tables.items():
            pivot_path = os.path.join(output_dir, f"metrics_summary_{metric}.csv")
            pivot_df.to_csv(pivot_path)
        print(f"Successfully saved all PIVOT summaries to {output_dir}")
    except Exception as e:
        print(f"\nWarning: Could not save summary files. Error: {e}")

    return summary_df, pivot_tables

all_rep_metrics = []

if 'all_model_registries' not in locals() or 'all_test_registries' not in locals():
    print("ERROR: 'all_model_registries' and 'all_test_registries' not found.")
    print("Please run the previous cell that creates these registry lists.")
else:
    N_REPETITIONS = len(all_model_registries)
    print(f"Found {N_REPETITIONS} data repetitions. Starting main loop for LSTM-AE...")

    for i in range(N_REPETITIONS):
        print(f"\n{'='*60}")
        print(f"          STARTING REPETITION {i + 1} / {N_REPETITIONS} (LSTM-AE)")
        print(f"{'='*60}\n")

        rep_output_dir = os.path.join(BASE_OUTPUT_DIR, f"Rep_{i+1}")
        os.makedirs(rep_output_dir, exist_ok=True)

        model_registry = all_model_registries[i]
        test_registry = all_test_registries[i]

        # --- 2. Training ---
        trained_models = {}
        print(f"--- Training Rep {i+1} ---")

        for key, bundle in tqdm(model_registry.items(), desc=f"Training (Rep {i+1})"):
            gran, tf = key
            dataset = bundle['loader'].dataset

            if hasattr(dataset, 'sequences'):
                raw_data = dataset.sequences
            else:
                raw_data = np.array([x[0].numpy() for x in dataset])

            train_tensor = torch.tensor(raw_data, dtype=torch.float32).to(DEVICE)
            n_total = train_tensor.size(0)
            if n_total == 0: continue

            model = LSTM_AE(N_FEATURES, SEQ_LENGTH, HIDDEN_DIM).to(DEVICE) # <-- LSTM Model
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=LR)

            model.train()
            loss_history = []
            for epoch in range(N_EPOCHS):
                epoch_indices = torch.randperm(n_total, device=DEVICE)
                epoch_loss = 0
                n_batches = 0
                for start in range(0, n_total, BATCH_SIZE):
                    end = min(start + BATCH_SIZE, n_total)
                    batch_idx = epoch_indices[start:end]
                    batch = train_tensor[batch_idx]

                    optimizer.zero_grad()
                    outputs = model(batch)
                    loss = criterion(outputs, batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                    n_batches += 1
                loss_history.append(epoch_loss / n_batches)

            trained_models[key] = { "model": model, "loss_history": loss_history }
            tqdm.write(f"[{gran} | {tf:<5}] (Rep {i+1}) Final Loss: {loss_history[-1]:.5f}")

        print(f"\n--- Scoring Rep {i+1} ---")
        scores_df = extract_anomaly_scores(trained_models, test_registry, DEVICE)
        scores_df.to_csv(os.path.join(rep_output_dir, "raw_scores.csv"), index=False)

        print(f"\n--- Analyzing Rep {i+1} ---")
        flat_results, pivot_tables = analyze_all_metrics(scores_df, output_dir=rep_output_dir)

        if pivot_tables is not None:
            all_rep_metrics.append(pivot_tables)

    print(f"\n\n{'='*60}")
    print("      ALL REPETITIONS COMPLETE (LSTM-AE) - FINAL AGGREGATED RESULTS")
    print(f"{'='*60}\n")

    if all_rep_metrics:
        all_metric_names = [
            "AUC",
            "F1_BestF1", "Precision_BestF1", "Recall_BestF1", "Accuracy_BestF1",
            "F1_ROC_TopLeft", "Precision_ROC_TopLeft", "Recall_ROC_TopLeft", "Accuracy_ROC_TopLeft",
            "F1_Recall_90", "Precision_Recall_90", "Recall_Recall_90", "Accuracy_Recall_90",
            "F1_FPR_10", "Precision_FPR_10", "Recall_FPR_10", "Accuracy_FPR_10"
        ]

        try:
            for metric in all_metric_names:
                metric_dfs = [rep[metric] for rep in all_rep_metrics if rep and metric in rep]
                if not metric_dfs:
                    continue

                concatenated_metric = pd.concat(metric_dfs)
                mean_metric = concatenated_metric.groupby(level=0).mean()
                std_metric = concatenated_metric.groupby(level=0).std()

                if "F1" in metric or "AUC" in metric:
                    print(f"--- Mean {metric} (across {len(metric_dfs)} Reps) ---")
                    print(mean_metric)
                    print(f"\n--- Std Dev {metric} (across {len(metric_dfs)} Reps) ---")
                    print(std_metric)

                mean_metric.to_csv(os.path.join(BASE_OUTPUT_DIR, f"aggregated_mean_{metric}.csv"))
                std_metric.to_csv(os.path.join(BASE_OUTPUT_DIR, f"aggregated_std_{metric}.csv"))

        except Exception as e:
            print(f"Error during final aggregation: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("No results were aggregated.")

Using device: cuda
Found 5 data repetitions. Starting main loop for LSTM-AE...

          STARTING REPETITION 1 / 5 (LSTM-AE)

--- Training Rep 1 ---


Training (Rep 1):   0%|          | 0/81 [00:00<?, ?it/s]

[10 | TF_0 ] (Rep 1) Final Loss: 0.00021
[10 | TF_1 ] (Rep 1) Final Loss: 0.00060
[10 | TF_2 ] (Rep 1) Final Loss: 0.00049
[10 | TF_3 ] (Rep 1) Final Loss: 0.00028
[10 | TF_4 ] (Rep 1) Final Loss: 0.00026
[10 | TF_5 ] (Rep 1) Final Loss: 0.00007
[10 | TF_6 ] (Rep 1) Final Loss: 0.00011
[10 | TF_7 ] (Rep 1) Final Loss: 0.00012
[10 | TF_8 ] (Rep 1) Final Loss: 0.00033
[10 | TF_9 ] (Rep 1) Final Loss: 0.00028
[2.5 | TF_0 ] (Rep 1) Final Loss: 0.00061
[2.5 | TF_1 ] (Rep 1) Final Loss: 0.00055
[2.5 | TF_10] (Rep 1) Final Loss: 0.00109
[2.5 | TF_11] (Rep 1) Final Loss: 0.00044
[2.5 | TF_12] (Rep 1) Final Loss: 0.00036
[2.5 | TF_13] (Rep 1) Final Loss: 0.00040
[2.5 | TF_14] (Rep 1) Final Loss: 0.00053
[2.5 | TF_15] (Rep 1) Final Loss: 0.00037
[2.5 | TF_16] (Rep 1) Final Loss: 0.00060
[2.5 | TF_17] (Rep 1) Final Loss: 0.00059
[2.5 | TF_18] (Rep 1) Final Loss: 0.00030
[2.5 | TF_19] (Rep 1) Final Loss: 0.00025
[2.5 | TF_2 ] (Rep 1) Final Loss: 0.00056
[2.5 | TF_20] (Rep 1) Final Loss: 0.00017
[2

Scoring Test Days:   0%|          | 0/81 [00:00<?, ?it/s]

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 1 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/LSTM-AE/Rep_1
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.748119  0.756850  0.820571  0.922581  0.780659  0.781481
Faulty (F+A)  0.600550  0.655660  0.604354  0.847312  0.647358  0.677778
Sensor 1      0.654080  0.581891  0.704955  0.774194  0.636464  0.611111
Sensor 2      0.649450  0.514699  0.684685  0.584946  0.564840  0.603704
Sensor 3      0.584925  0.583878  0.526276  0.688172  0.646543  0.811111
Sensor 4      0.608796  0.570369  0.599099  0.744086  0.591031  0.411111

--- F1_BestF1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        2

...

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 4 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/LSTM-AE/Rep_4
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.752749  0.759877  0.792042  0.938710  0.791389  0.800000
Faulty (F+A)  0.629196  0.616646  0.613363  0.867742  0.672864  0.681481
Sensor 1      0.653501  0.576180  0.674925  0.835484  0.636204  0.688889
Sensor 2      0.626881  0.520369  0.573574  0.548387  0.532616  0.651852
Sensor 3      0.555700  0.541959  0.574324  0.747312  0.646249  0.696296
Sensor 4      0.590567  0.553037  0.569820  0.626882  0.614644  0.503704

--- F1_BestF1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        2

Training (Rep 5):   0%|          | 0/81 [00:00<?, ?it/s]

[10 | TF_0 ] (Rep 5) Final Loss: 0.00086
[10 | TF_1 ] (Rep 5) Final Loss: 0.00060
[10 | TF_2 ] (Rep 5) Final Loss: 0.00053
[10 | TF_3 ] (Rep 5) Final Loss: 0.00027
[10 | TF_4 ] (Rep 5) Final Loss: 0.00026
[10 | TF_5 ] (Rep 5) Final Loss: 0.00006
[10 | TF_6 ] (Rep 5) Final Loss: 0.00010
[10 | TF_7 ] (Rep 5) Final Loss: 0.00012
[10 | TF_8 ] (Rep 5) Final Loss: 0.00020
[10 | TF_9 ] (Rep 5) Final Loss: 0.00039
[2.5 | TF_0 ] (Rep 5) Final Loss: 0.00092
[2.5 | TF_1 ] (Rep 5) Final Loss: 0.00149
[2.5 | TF_10] (Rep 5) Final Loss: 0.00105
[2.5 | TF_11] (Rep 5) Final Loss: 0.00062
[2.5 | TF_12] (Rep 5) Final Loss: 0.00051
[2.5 | TF_13] (Rep 5) Final Loss: 0.00045
[2.5 | TF_14] (Rep 5) Final Loss: 0.00041
[2.5 | TF_15] (Rep 5) Final Loss: 0.00037
[2.5 | TF_16] (Rep 5) Final Loss: 0.00046
[2.5 | TF_17] (Rep 5) Final Loss: 0.00038
[2.5 | TF_18] (Rep 5) Final Loss: 0.00028
[2.5 | TF_19] (Rep 5) Final Loss: 0.00032
[2.5 | TF_2 ] (Rep 5) Final Loss: 0.00101
[2.5 | TF_20] (Rep 5) Final Loss: 0.00014
[2

Scoring Test Days:   0%|          | 0/81 [00:00<?, ?it/s]

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 5 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/LSTM-AE/Rep_5
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.717882  0.757516  0.812312  0.900000  0.770222  0.770370
Faulty (F+A)  0.570312  0.654883  0.684685  0.813978  0.631898  0.696296
Sensor 1      0.582465  0.597306  0.681682  0.730108  0.611709  0.740741
Sensor 2      0.628762  0.510967  0.519520  0.546237  0.552381  0.603704
Sensor 3      0.574653  0.592998  0.603604  0.725806  0.615362  0.762963
Sensor 4      0.603009  0.582072  0.708709  0.730108  0.596086  0.429630

--- F1_BestF1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        2

# **GRU-AE**

In [ ]:
N_FEATURES = len(FEATURE_COLS)
HIDDEN_DIM = 16
N_EPOCHS = 50
BATCH_SIZE = 512
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_OUTPUT_DIR = output_metrics_dir + "GRU-AE"

print(f"Using device: {DEVICE}")

class GRU_AE(nn.Module):
    def __init__(self, n_features, seq_len, hidden_dim):
        super(GRU_AE, self).__init__()
        self.seq_len = seq_len
        self.n_features = n_features
        self.hidden_dim = hidden_dim

        self.encoder = nn.GRU(
            input_size=n_features,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.decoder = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.output_layer = nn.Linear(hidden_dim, n_features)

    def forward(self, x):
        _, hidden = self.encoder(x)
        decoder_input = hidden.squeeze(0).unsqueeze(1).repeat(1, self.seq_len, 1)
        dec_out, _ = self.decoder(decoder_input)
        reconstruction = self.output_layer(dec_out)
        return reconstruction

def extract_anomaly_scores(trained_models_dict, test_registry_dict, device):
    results_list = []
    print(f"{'='*20} STARTING SCORE EXTRACTION {'='*20}")

    for key, test_days in tqdm(test_registry_dict.items(), desc="Scoring Test Days", leave=False):
        gran, tf = key
        if key not in trained_models_dict:
            continue

        bundle = trained_models_dict[key]
        model = bundle['model'].to(device).eval()
        for day_idx, day_obj in enumerate(test_days):
            inputs = day_obj['data'].to(device)
            if len(inputs) == 0:
                continue
            with torch.no_grad():
                reconstructions = model(inputs)
                mse_per_seq = torch.mean((inputs - reconstructions)**2, dim=(1, 2))
                day_score = torch.mean(mse_per_seq).item()
            results_list.append({
                "granularity": gran, "time_frame": tf, "day_index": day_idx,
                "ad_score": day_score, "true_label": day_obj['label'],
                "category": day_obj['category']
            })
    df_results = pd.DataFrame(results_list)
    print(f"Extraction Complete. Processed {len(df_results)} days.")
    return df_results

def analyze_all_metrics(df_results, output_dir: str):
    """
    Analyzes AUC and finds the optimal F1, Precision, Recall, and Accuracy
    by finding the best empirical threshold.
    """
    if df_results.empty:
        print("DataFrame is empty.")
        return None, None

    os.makedirs(output_dir, exist_ok=True)
    roc_data_dir = os.path.join(output_dir, "roc_curve_data")
    os.makedirs(roc_data_dir, exist_ok=True)
    print(f"Metrics and ROC data will be saved to {output_dir}")

    subgroups = {
        "Fault Free":   lambda x: "FF" in x, "Faulty (F+A)": lambda x: "FA" in x,
        "Sensor 1":     lambda x: "sensor_0" in x, "Sensor 2":     lambda x: "sensor_1" in x,
        "Sensor 3":     lambda x: "sensor_2" in x, "Sensor 4":     lambda x: "sensor_3" in x,
    }
    metrics_list = []
    unique_grans = sorted(df_results['granularity'].unique())

    print(f"{'='*20} DETAILED METRIC ANALYSIS BY GRANULARITY {'='*20}")
    for group_name, filter_func in subgroups.items():
        for gran in unique_grans:
            subset = df_results[
                (df_results['granularity'] == gran) &
                (df_results['category'].apply(filter_func))
            ]
            if len(subset) == 0:
                continue

            y_true, y_scores = subset['true_label'], subset['ad_score']

            auc, f1, prec, rec, acc, best_thresh = [np.nan] * 6

            if len(np.unique(y_true)) > 1:
                auc = roc_auc_score(y_true, y_scores)

                precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_true, y_scores)

                f1_scores = np.divide(
                    2 * precision_curve * recall_curve,
                    precision_curve + recall_curve,
                    out=np.zeros_like(precision_curve),
                    where=(precision_curve + recall_curve) != 0
                )

                best_idx = np.argmax(f1_scores)
                best_thresh = thresholds_pr[min(best_idx, len(thresholds_pr)-1)]

                y_pred = (y_scores >= best_thresh).astype(int)

                f1 = f1_score(y_true, y_pred, zero_division=0)
                prec = precision_score(y_true, y_pred, zero_division=0)
                rec = recall_score(y_true, y_pred, zero_division=0)
                acc = accuracy_score(y_true, y_pred)

                try:
                    fpr, tpr, thresholds_roc = roc_curve(y_true, y_scores)
                    roc_data_df = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds_roc})
                    safe_group_name = group_name.replace(" ", "_").replace("+", "").replace("(", "").replace(")", "")
                    file_name = f"roc_{safe_group_name}_{str(gran)}.csv"
                    roc_data_df.to_csv(os.path.join(roc_data_dir, file_name), index=False)
                except Exception as e:
                    print(f"Warning: Could not save ROC data for {group_name}/{gran}. Error: {e}")

            metrics_list.append({
                "Dataset": group_name, "Granularity": gran,
                "AUC": auc, "F1": f1, "Precision": prec, "Recall": rec, "Accuracy": acc,
                "Best_Threshold": best_thresh,
                "Total Days": len(subset), "Normal": sum(y_true == 0), "Anom": sum(y_true == 1)
            })

    summary_df = pd.DataFrame(metrics_list)

    metrics_to_pivot = ["AUC", "F1", "Precision", "Recall", "Accuracy"]
    pivot_tables = {}

    for metric in metrics_to_pivot:
        pivot = summary_df.pivot(index="Dataset", columns="Granularity", values=metric)
        pivot_tables[metric] = pivot

        print(f"\n--- {metric} Matrix (Rows=Dataset, Cols=Granularity) ---")
        print(pivot)

    try:
        summary_path = os.path.join(output_dir, "metrics_summary_long.csv")
        summary_df.to_csv(summary_path, index=False)
        print(f"\nSuccessfully saved LONG summary to {output_dir}")

        for metric, pivot_df in pivot_tables.items():
            pivot_path = os.path.join(output_dir, f"metrics_summary_{metric}.csv")
            pivot_df.to_csv(pivot_path)
        print(f"Successfully saved all PIVOT summaries to {output_dir}")
    except Exception as e:
        print(f"\nWarning: Could not save summary files. Error: {e}")

    return summary_df, pivot_tables

all_rep_metrics = []

if 'all_model_registries' not in locals() or 'all_test_registries' not in locals():
    print("ERROR: 'all_model_registries' and 'all_test_registries' not found.")
    print("Please run the previous cell that creates these registry lists.")
else:
    N_REPETITIONS = len(all_model_registries)
    print(f"Found {N_REPETITIONS} data repetitions. Starting main loop for GRU-AE...")

    for i in range(N_REPETITIONS):
        print(f"\n{'='*60}")
        print(f"          STARTING REPETITION {i + 1} / {N_REPETITIONS} (GRU-AE)")
        print(f"{'='*60}\n")

        rep_output_dir = os.path.join(BASE_OUTPUT_DIR, f"Rep_{i+1}")
        os.makedirs(rep_output_dir, exist_ok=True)

        model_registry = all_model_registries[i]
        test_registry = all_test_registries[i]

        trained_models = {}
        print(f"--- Training Rep {i+1} ---")

        for key, bundle in tqdm(model_registry.items(), desc=f"Training (Rep {i+1})"):
            gran, tf = key
            dataset = bundle['loader'].dataset

            if hasattr(dataset, 'sequences'):
                raw_data = dataset.sequences
            else:
                raw_data = np.array([x[0].numpy() for x in dataset])

            train_tensor = torch.tensor(raw_data, dtype=torch.float32).to(DEVICE)
            n_total = train_tensor.size(0)
            if n_total == 0: continue

            model = GRU_AE(N_FEATURES, SEQ_LENGTH, HIDDEN_DIM).to(DEVICE) # <--- GRU MODEL
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=LR)

            model.train()
            loss_history = []
            for epoch in range(N_EPOCHS):
                epoch_indices = torch.randperm(n_total, device=DEVICE)
                epoch_loss = 0
                n_batches = 0
                for start in range(0, n_total, BATCH_SIZE):
                    end = min(start + BATCH_SIZE, n_total)
                    batch_idx = epoch_indices[start:end]
                    batch = train_tensor[batch_idx]

                    optimizer.zero_grad()
                    outputs = model(batch)
                    loss = criterion(outputs, batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                    n_batches += 1
                loss_history.append(epoch_loss / n_batches)

            trained_models[key] = { "model": model, "loss_history": loss_history }
            tqdm.write(f"[{gran} | {tf:<5}] (Rep {i+1}) Final Loss: {loss_history[-1]:.5f}")

        print(f"\n--- Scoring Rep {i+1} ---")
        scores_df = extract_anomaly_scores(trained_models, test_registry, DEVICE)
        scores_df.to_csv(os.path.join(rep_output_dir, "raw_scores.csv"), index=False)

        print(f"\n--- Analyzing Rep {i+1} ---")
        flat_results, pivot_tables = analyze_all_metrics(scores_df, output_dir=rep_output_dir)

        if pivot_tables is not None:
            all_rep_metrics.append(pivot_tables)

    print(f"\n\n{'='*60}")
    print("      ALL REPETITIONS COMPLETE (GRU-AE) - FINAL AGGREGATED RESULTS")
    print(f"{'='*60}\n")

    if all_rep_metrics:
        try:
            for metric in ["AUC", "F1", "Precision", "Recall", "Accuracy"]:
                metric_dfs = [rep[metric] for rep in all_rep_metrics if rep and metric in rep]

                if not metric_dfs:
                    print(f"No results found for metric: {metric}")
                    continue

                concatenated_metric = pd.concat(metric_dfs)

                mean_metric = concatenated_metric.groupby(level=0).mean()
                std_metric = concatenated_metric.groupby(level=0).std()

                print(f"--- Mean {metric} (across {len(metric_dfs)} Reps) ---")
                print(mean_metric)
                print(f"\n--- Std Dev {metric} (across {len(metric_dfs)} Reps) ---")
                print(std_metric)

                mean_metric.to_csv(os.path.join(BASE_OUTPUT_DIR, f"aggregated_mean_{metric}.csv"))
                std_metric.to_csv(os.path.join(BASE_OUTPUT_DIR, f"aggregated_std_{metric}.csv"))

        except Exception as e:
            print(f"Error during final aggregation: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("No results were aggregated.")

Using device: cuda
Found 5 data repetitions. Starting main loop for GRU-AE...

          STARTING REPETITION 1 / 5 (GRU-AE)

--- Training Rep 1 ---


Training (Rep 1):   0%|          | 0/81 [00:00<?, ?it/s]

[10 | TF_0 ] (Rep 1) Final Loss: 0.00021
[10 | TF_1 ] (Rep 1) Final Loss: 0.00069
[10 | TF_2 ] (Rep 1) Final Loss: 0.00053
[10 | TF_3 ] (Rep 1) Final Loss: 0.00028
[10 | TF_4 ] (Rep 1) Final Loss: 0.00027
[10 | TF_5 ] (Rep 1) Final Loss: 0.00009
[10 | TF_6 ] (Rep 1) Final Loss: 0.00014
[10 | TF_7 ] (Rep 1) Final Loss: 0.00012
[10 | TF_8 ] (Rep 1) Final Loss: 0.00036
[10 | TF_9 ] (Rep 1) Final Loss: 0.00029
[2.5 | TF_0 ] (Rep 1) Final Loss: 0.00059
[2.5 | TF_1 ] (Rep 1) Final Loss: 0.00042
[2.5 | TF_10] (Rep 1) Final Loss: 0.00117
[2.5 | TF_11] (Rep 1) Final Loss: 0.00045
[2.5 | TF_12] (Rep 1) Final Loss: 0.00053
[2.5 | TF_13] (Rep 1) Final Loss: 0.00062
[2.5 | TF_14] (Rep 1) Final Loss: 0.00061
[2.5 | TF_15] (Rep 1) Final Loss: 0.00060
[2.5 | TF_16] (Rep 1) Final Loss: 0.00064
[2.5 | TF_17] (Rep 1) Final Loss: 0.00073
[2.5 | TF_18] (Rep 1) Final Loss: 0.00037
[2.5 | TF_19] (Rep 1) Final Loss: 0.00043
[2.5 | TF_2 ] (Rep 1) Final Loss: 0.00036
[2.5 | TF_20] (Rep 1) Final Loss: 0.00033
[2

Scoring Test Days:   0%|          | 0/81 [00:00<?, ?it/s]

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 1 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/GRU-AE/Rep_1
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.771267  0.740486  0.804054  0.909677  0.785421  0.796296
Faulty (F+A)  0.631800  0.655236  0.578829  0.862366  0.651924  0.714815
Sensor 1      0.648003  0.571661  0.673423  0.815054  0.641944  0.655556
Sensor 2      0.620515  0.506184  0.735736  0.589247  0.569569  0.625926
Sensor 3      0.593605  0.579923  0.533784  0.709677  0.649870  0.811111
Sensor 4      0.609086  0.570410  0.578078  0.795699  0.595858  0.648148

--- F1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25       

...

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 4 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/GRU-AE/Rep_4
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.753328  0.748295  0.781532  0.932258  0.808415  0.844444
Faulty (F+A)  0.624566  0.629439  0.611862  0.880645  0.689367  0.681481
Sensor 1      0.652344  0.581628  0.638889  0.835484  0.643379  0.703704
Sensor 2      0.663050  0.504086  0.534535  0.526882  0.576843  0.600000
Sensor 3      0.557726  0.547851  0.571321  0.755914  0.651402  0.711111
Sensor 4      0.587095  0.565143  0.623874  0.723656  0.626060  0.637037

--- F1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25       

Training (Rep 5):   0%|          | 0/81 [00:00<?, ?it/s]

[10 | TF_0 ] (Rep 5) Final Loss: 0.00087
[10 | TF_1 ] (Rep 5) Final Loss: 0.00064
[10 | TF_2 ] (Rep 5) Final Loss: 0.00053
[10 | TF_3 ] (Rep 5) Final Loss: 0.00028
[10 | TF_4 ] (Rep 5) Final Loss: 0.00028
[10 | TF_5 ] (Rep 5) Final Loss: 0.00007
[10 | TF_6 ] (Rep 5) Final Loss: 0.00013
[10 | TF_7 ] (Rep 5) Final Loss: 0.00013
[10 | TF_8 ] (Rep 5) Final Loss: 0.00020
[10 | TF_9 ] (Rep 5) Final Loss: 0.00042
[2.5 | TF_0 ] (Rep 5) Final Loss: 0.00096
[2.5 | TF_1 ] (Rep 5) Final Loss: 0.00143
[2.5 | TF_10] (Rep 5) Final Loss: 0.00110
[2.5 | TF_11] (Rep 5) Final Loss: 0.00046
[2.5 | TF_12] (Rep 5) Final Loss: 0.00069
[2.5 | TF_13] (Rep 5) Final Loss: 0.00044
[2.5 | TF_14] (Rep 5) Final Loss: 0.00053
[2.5 | TF_15] (Rep 5) Final Loss: 0.00064
[2.5 | TF_16] (Rep 5) Final Loss: 0.00054
[2.5 | TF_17] (Rep 5) Final Loss: 0.00055
[2.5 | TF_18] (Rep 5) Final Loss: 0.00029
[2.5 | TF_19] (Rep 5) Final Loss: 0.00029
[2.5 | TF_2 ] (Rep 5) Final Loss: 0.00092
[2.5 | TF_20] (Rep 5) Final Loss: 0.00022
[2

Scoring Test Days:   0%|          | 0/81 [00:00<?, ?it/s]

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 5 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/GRU-AE/Rep_5
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.727431  0.752976  0.798799  0.891398  0.767613  0.807407
Faulty (F+A)  0.575521  0.675928  0.662162  0.820430  0.633725  0.718519
Sensor 1      0.593171  0.606003  0.668168  0.730108  0.610209  0.725926
Sensor 2      0.611545  0.539800  0.519520  0.595699  0.524070  0.625926
Sensor 3      0.575521  0.600646  0.599099  0.723656  0.614905  0.807407
Sensor 4      0.599826  0.586713  0.687688  0.702151  0.577886  0.592593

--- F1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25       

# **1D-CNN-AE**

In [ ]:
N_FEATURES = len(FEATURE_COLS)
N_EPOCHS = 50
BATCH_SIZE = 2048
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_OUTPUT_DIR = output_metrics_dir + "CNN-AE"

print(f"Using device: {DEVICE}")

class CNN_AE(nn.Module):
    def __init__(self, n_features, seq_len):
        super(CNN_AE, self).__init__()
        self.seq_len = seq_len

        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=n_features, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2), # 10 -> 5

            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)  # 5 -> 2
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=2, stride=2), # 2 -> 4
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=n_features, kernel_size=2, stride=2)  # 4 -> 8
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        decoded = nn.functional.interpolate(decoded, size=self.seq_len, mode='linear', align_corners=False)

        decoded = decoded.permute(0, 2, 1)
        return decoded

def extract_anomaly_scores(trained_models_dict, test_registry_dict, device):
    results_list = []
    print(f"{'='*20} STARTING SCORE EXTRACTION {'='*20}")

    for key, test_days in tqdm(test_registry_dict.items(), desc="Scoring Test Days", leave=False):
        gran, tf = key
        if key not in trained_models_dict:
            continue

        bundle = trained_models_dict[key]
        model = bundle['model'].to(device).eval()
        for day_idx, day_obj in enumerate(test_days):
            inputs = day_obj['data'].to(device)
            if len(inputs) == 0:
                continue
            with torch.no_grad():
                reconstructions = model(inputs)
                mse_per_seq = torch.mean((inputs - reconstructions)**2, dim=(1, 2))
                day_score = torch.mean(mse_per_seq).item()
            results_list.append({
                "granularity": gran, "time_frame": tf, "day_index": day_idx,
                "ad_score": day_score, "true_label": day_obj['label'],
                "category": day_obj['category']
            })
    df_results = pd.DataFrame(results_list)
    print(f"Extraction Complete. Processed {len(df_results)} days.")
    return df_results

def analyze_all_metrics(df_results, output_dir: str):
    """
    Analyzes AUC and finds the optimal F1, Precision, Recall, and Accuracy
    by finding the best empirical threshold.
    """
    if df_results.empty:
        print("DataFrame is empty.")
        return None, None

    os.makedirs(output_dir, exist_ok=True)
    roc_data_dir = os.path.join(output_dir, "roc_curve_data")
    os.makedirs(roc_data_dir, exist_ok=True)
    print(f"Metrics and ROC data will be saved to {output_dir}")

    subgroups = {
        "Fault Free":   lambda x: "FF" in x, "Faulty (F+A)": lambda x: "FA" in x,
        "Sensor 1":     lambda x: "sensor_0" in x, "Sensor 2":     lambda x: "sensor_1" in x,
        "Sensor 3":     lambda x: "sensor_2" in x, "Sensor 4":     lambda x: "sensor_3" in x,
    }
    metrics_list = []
    unique_grans = sorted(df_results['granularity'].unique())

    print(f"{'='*20} DETAILED METRIC ANALYSIS BY GRANULARITY {'='*20}")
    for group_name, filter_func in subgroups.items():
        for gran in unique_grans:
            subset = df_results[
                (df_results['granularity'] == gran) &
                (df_results['category'].apply(filter_func))
            ]
            if len(subset) == 0:
                continue

            y_true, y_scores = subset['true_label'], subset['ad_score']

            auc, f1, prec, rec, acc, best_thresh = [np.nan] * 6

            if len(np.unique(y_true)) > 1:
                auc = roc_auc_score(y_true, y_scores)

                precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_true, y_scores)

                f1_scores = np.divide(
                    2 * precision_curve * recall_curve,
                    precision_curve + recall_curve,
                    out=np.zeros_like(precision_curve),
                    where=(precision_curve + recall_curve) != 0
                )

                best_idx = np.argmax(f1_scores)
                best_thresh = thresholds_pr[min(best_idx, len(thresholds_pr)-1)]

                y_pred = (y_scores >= best_thresh).astype(int)

                f1 = f1_score(y_true, y_pred, zero_division=0)
                prec = precision_score(y_true, y_pred, zero_division=0)
                rec = recall_score(y_true, y_pred, zero_division=0)
                acc = accuracy_score(y_true, y_pred)

                try:
                    fpr, tpr, thresholds_roc = roc_curve(y_true, y_scores)
                    roc_data_df = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds_roc})
                    safe_group_name = group_name.replace(" ", "_").replace("+", "").replace("(", "").replace(")", "")
                    file_name = f"roc_{safe_group_name}_{str(gran)}.csv"
                    roc_data_df.to_csv(os.path.join(roc_data_dir, file_name), index=False)
                except Exception as e:
                    print(f"Warning: Could not save ROC data for {group_name}/{gran}. Error: {e}")

            metrics_list.append({
                "Dataset": group_name, "Granularity": gran,
                "AUC": auc, "F1": f1, "Precision": prec, "Recall": rec, "Accuracy": acc,
                "Best_Threshold": best_thresh,
                "Total Days": len(subset), "Normal": sum(y_true == 0), "Anom": sum(y_true == 1)
            })

    summary_df = pd.DataFrame(metrics_list)

    metrics_to_pivot = ["AUC", "F1", "Precision", "Recall", "Accuracy"]
    pivot_tables = {}

    for metric in metrics_to_pivot:
        pivot = summary_df.pivot(index="Dataset", columns="Granularity", values=metric)
        pivot_tables[metric] = pivot

        print(f"\n--- {metric} Matrix (Rows=Dataset, Cols=Granularity) ---")
        print(pivot)

    try:
        summary_path = os.path.join(output_dir, "metrics_summary_long.csv")
        summary_df.to_csv(summary_path, index=False)
        print(f"\nSuccessfully saved LONG summary to {output_dir}")

        for metric, pivot_df in pivot_tables.items():
            pivot_path = os.path.join(output_dir, f"metrics_summary_{metric}.csv")
            pivot_df.to_csv(pivot_path)
        print(f"Successfully saved all PIVOT summaries to {output_dir}")
    except Exception as e:
        print(f"\nWarning: Could not save summary files. Error: {e}")

    return summary_df, pivot_tables

all_rep_metrics = []

if 'all_model_registries' not in locals() or 'all_test_registries' not in locals():
    print("ERROR: 'all_model_registries' and 'all_test_registries' not found.")
    print("Please run the previous cell that creates these registry lists.")
else:
    N_REPETITIONS = len(all_model_registries)
    print(f"Found {N_REPETITIONS} data repetitions. Starting main loop for CNN-AE...")

    for i in range(N_REPETITIONS):
        print(f"\n{'='*60}")
        print(f"          STARTING REPETITION {i + 1} / {N_REPETITIONS} (CNN-AE)")
        print(f"{'='*60}\n")

        rep_output_dir = os.path.join(BASE_OUTPUT_DIR, f"Rep_{i+1}")
        os.makedirs(rep_output_dir, exist_ok=True)

        model_registry = all_model_registries[i]
        test_registry = all_test_registries[i]

        trained_models = {}
        print(f"--- Training Rep {i+1} ---")

        for key, bundle in tqdm(model_registry.items(), desc=f"Training (Rep {i+1})"):
            gran, tf = key
            dataset = bundle['loader'].dataset

            if hasattr(dataset, 'sequences'):
                raw_data = dataset.sequences
            else:
                raw_data = np.array([x[0].numpy() for x in dataset])

            train_tensor = torch.tensor(raw_data, dtype=torch.float32).to(DEVICE)
            n_total = train_tensor.size(0)
            if n_total == 0: continue

            model = CNN_AE(N_FEATURES, SEQ_LENGTH).to(DEVICE) # <--- CNN MODEL
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=LR)

            model.train()
            loss_history = []
            for epoch in range(N_EPOCHS):
                epoch_indices = torch.randperm(n_total, device=DEVICE)
                epoch_loss = 0
                n_batches = 0
                for start in range(0, n_total, BATCH_SIZE):
                    end = min(start + BATCH_SIZE, n_total)
                    batch_idx = epoch_indices[start:end]
                    batch = train_tensor[batch_idx]

                    optimizer.zero_grad()
                    outputs = model(batch)
                    loss = criterion(outputs, batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                    n_batches += 1
                loss_history.append(epoch_loss / n_batches)

            trained_models[key] = { "model": model, "loss_history": loss_history }
            tqdm.write(f"[{gran} | {tf:<5}] (Rep {i+1}) Final Loss: {loss_history[-1]:.5f}")

        print(f"\n--- Scoring Rep {i+1} ---")
        scores_df = extract_anomaly_scores(trained_models, test_registry, DEVICE)
        scores_df.to_csv(os.path.join(rep_output_dir, "raw_scores.csv"), index=False)

        print(f"\n--- Analyzing Rep {i+1} ---")
        flat_results, pivot_tables = analyze_all_metrics(scores_df, output_dir=rep_output_dir)

        if pivot_tables is not None:
            all_rep_metrics.append(pivot_tables)

    print(f"\n\n{'='*60}")
    print("      ALL REPETITIONS COMPLETE (CNN-AE) - FINAL AGGREGATED RESULTS")
    print(f"{'='*60}\n")

    if all_rep_metrics:
        try:
            for metric in ["AUC", "F1", "Precision", "Recall", "Accuracy"]:
                metric_dfs = [rep[metric] for rep in all_rep_metrics if rep and metric in rep]

                if not metric_dfs:
                    print(f"No results found for metric: {metric}")
                    continue

                concatenated_metric = pd.concat(metric_dfs)

                mean_metric = concatenated_metric.groupby(level=0).mean()
                std_metric = concatenated_metric.groupby(level=0).std()

                print(f"--- Mean {metric} (across {len(metric_dfs)} Reps) ---")
                print(mean_metric)
                print(f"\n--- Std Dev {metric} (across {len(metric_dfs)} Reps) ---")
                print(std_metric)

                mean_metric.to_csv(os.path.join(BASE_OUTPUT_DIR, f"aggregated_mean_{metric}.csv"))
                std_metric.to_csv(os.path.join(BASE_OUTPUT_DIR, f"aggregated_std_{metric}.csv"))

        except Exception as e:
            print(f"Error during final aggregation: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("No results were aggregated.")

Using device: cuda
Found 5 data repetitions. Starting main loop for CNN-AE...

          STARTING REPETITION 1 / 5 (CNN-AE)

--- Training Rep 1 ---


Training (Rep 1):   0%|          | 0/81 [00:00<?, ?it/s]

[10 | TF_0 ] (Rep 1) Final Loss: 0.00014
[10 | TF_1 ] (Rep 1) Final Loss: 0.00024
[10 | TF_2 ] (Rep 1) Final Loss: 0.00039
[10 | TF_3 ] (Rep 1) Final Loss: 0.00017
[10 | TF_4 ] (Rep 1) Final Loss: 0.00019
[10 | TF_5 ] (Rep 1) Final Loss: 0.00007
[10 | TF_6 ] (Rep 1) Final Loss: 0.00015
[10 | TF_7 ] (Rep 1) Final Loss: 0.00007
[10 | TF_8 ] (Rep 1) Final Loss: 0.00024
[10 | TF_9 ] (Rep 1) Final Loss: 0.00014
[2.5 | TF_0 ] (Rep 1) Final Loss: 0.00042
[2.5 | TF_1 ] (Rep 1) Final Loss: 0.00081
[2.5 | TF_10] (Rep 1) Final Loss: 0.00067
[2.5 | TF_11] (Rep 1) Final Loss: 0.00048
[2.5 | TF_12] (Rep 1) Final Loss: 0.00316
[2.5 | TF_13] (Rep 1) Final Loss: 0.00268
[2.5 | TF_14] (Rep 1) Final Loss: 0.00039
[2.5 | TF_15] (Rep 1) Final Loss: 0.00030
[2.5 | TF_16] (Rep 1) Final Loss: 0.00047
[2.5 | TF_17] (Rep 1) Final Loss: 0.00142
[2.5 | TF_18] (Rep 1) Final Loss: 0.00038
[2.5 | TF_19] (Rep 1) Final Loss: 0.00011
[2.5 | TF_2 ] (Rep 1) Final Loss: 0.00228
[2.5 | TF_20] (Rep 1) Final Loss: 0.00161
[2

Scoring Test Days:   0%|          | 0/81 [00:00<?, ?it/s]

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 1 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/CNN-AE/Rep_1
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.809462  0.731870  0.855105  0.926882  0.759002  0.722222
Faulty (F+A)  0.672020  0.617444  0.628378  0.819355  0.605088  0.648148
Sensor 1      0.730758  0.584130  0.704955  0.731183  0.613960  0.640741
Sensor 2      0.643374  0.530468  0.639640  0.586022  0.559817  0.600000
Sensor 3      0.641059  0.573366  0.545796  0.670968  0.612035  0.729630
Sensor 4      0.634549  0.544784  0.588589  0.675269  0.543542  0.455556

--- F1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25       

...

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 4 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/CNN-AE/Rep_4
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.782841  0.779409  0.813063  0.904301  0.795499  0.814815
Faulty (F+A)  0.633536  0.628471  0.694444  0.747312  0.681409  0.614815
Sensor 1      0.689091  0.624082  0.728979  0.687097  0.644031  0.666667
Sensor 2      0.639034  0.558333  0.573574  0.479570  0.566601  0.625926
Sensor 3      0.576823  0.557678  0.626877  0.676344  0.671168  0.688889
Sensor 4      0.570023  0.552714  0.520270  0.472043  0.616145  0.392593

--- F1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25       

Training (Rep 5):   0%|          | 0/81 [00:00<?, ?it/s]

[10 | TF_0 ] (Rep 5) Final Loss: 0.00065
[10 | TF_1 ] (Rep 5) Final Loss: 0.00034
[10 | TF_2 ] (Rep 5) Final Loss: 0.00024
[10 | TF_3 ] (Rep 5) Final Loss: 0.00013
[10 | TF_4 ] (Rep 5) Final Loss: 0.00012
[10 | TF_5 ] (Rep 5) Final Loss: 0.00003
[10 | TF_6 ] (Rep 5) Final Loss: 0.00006
[10 | TF_7 ] (Rep 5) Final Loss: 0.00007
[10 | TF_8 ] (Rep 5) Final Loss: 0.00010
[10 | TF_9 ] (Rep 5) Final Loss: 0.00025
[2.5 | TF_0 ] (Rep 5) Final Loss: 0.00116
[2.5 | TF_1 ] (Rep 5) Final Loss: 0.00104
[2.5 | TF_10] (Rep 5) Final Loss: 0.00074
[2.5 | TF_11] (Rep 5) Final Loss: 0.00075
[2.5 | TF_12] (Rep 5) Final Loss: 0.00071
[2.5 | TF_13] (Rep 5) Final Loss: 0.00088
[2.5 | TF_14] (Rep 5) Final Loss: 0.00035
[2.5 | TF_15] (Rep 5) Final Loss: 0.00044
[2.5 | TF_16] (Rep 5) Final Loss: 0.00029
[2.5 | TF_17] (Rep 5) Final Loss: 0.00072
[2.5 | TF_18] (Rep 5) Final Loss: 0.00022
[2.5 | TF_19] (Rep 5) Final Loss: 0.00016
[2.5 | TF_2 ] (Rep 5) Final Loss: 0.00255
[2.5 | TF_20] (Rep 5) Final Loss: 0.00020
[2

Scoring Test Days:   0%|          | 0/81 [00:00<?, ?it/s]

Extraction Complete. Processed 5754 days.

--- Analyzing Rep 5 ---
Metrics and ROC data will be saved to /content/drive/MyDrive/AD_SMART_CITY/Output/Metrics/CNN-AE/Rep_5
==================== DETAILED METRIC ANALYSIS BY GRANULARITY ====================

--- AUC Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25         5        50
Dataset                                                                 
Fault Free    0.701968  0.726423  0.803303  0.884946  0.752674  0.748148
Faulty (F+A)  0.573785  0.610412  0.632132  0.822581  0.611742  0.629630
Sensor 1      0.605035  0.605922  0.660661  0.687097  0.586660  0.688889
Sensor 2      0.587240  0.499788  0.585586  0.535484  0.553490  0.629630
Sensor 3      0.589699  0.568160  0.609610  0.803226  0.621233  0.725926
Sensor 4      0.588542  0.550999  0.624625  0.700000  0.592237  0.303704

--- F1 Matrix (Rows=Dataset, Cols=Granularity) ---
Granularity         10       2.5        20        25       